In [ ]:
20

In [ ]:
20

In [11]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [12]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [13]:


from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_VAE_separated import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

2023-01-11 18:32:44,283 (register:12)WARNING: Key laplacian_trainer is already pre-defined, overwritten.


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [14]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_MI.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 0
cfg.params.diff_importance = 1  # 1
cfg.params.mine_lr = 0.01
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 0.
cfg.params.recon_importance = 0.
cfg.params.kld_importance = 0.


#### configure other options

In [15]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test_no_KLD'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 2
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'


In [16]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [17]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [18]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


In [19]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-11 18:32:45,146 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-11 18:32:45,147 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-11 18:32:45,148 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_MINE_VAE_test_no_KLD_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230111183245


In [ ]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_VAE_Separated_client import LaplacianDomainSeparation1MINE_Separated_Client
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINE_Separated_Client,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

2023-01-11 18:32:46,888 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-11 18:32:46,911 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 2
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  save_dir: test_dir
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: ooxx
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transf

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight
encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.c

2023-01-11 18:32:48,550 (laplacian_with_domain_separation_1MINE_VAE_Separated_client:144)INFO: {'Role': 'Client #2', 'Round': 0, 'Results_raw': {'train_total': 512, 'train_avg_loss': 1.016006, 'train_loss': 520.194908, 'train_imp_ratio': -3733.983529}}


mi_local_global: 0.0015282109379768372
mi_global_fixed: -0.0026334673166275024
rec_loss: 0.9867874383926392
kld_loss: 1.7501764297485352
in train
mi_local_global: 0.0025653764605522156
mi_global_fixed: 0.0035875141620635986
rec_loss: 1.0027194023132324
kld_loss: 1.8420288562774658
mi_local_global: 0.0014493092894554138
mi_global_fixed: 0.0014196038246154785
rec_loss: 1.0012685060501099
kld_loss: 1.8698410987854004
mi_local_global: -0.0005914941430091858
mi_global_fixed: -0.0022449493408203125
rec_loss: 1.006597638130188
kld_loss: 1.9020382165908813
mi_local_global: 0.0012808740139007568
mi_global_fixed: 0.0023077204823493958
rec_loss: 1.0025221109390259
kld_loss: 1.8447434902191162
mi_local_global: 0.0005629956722259521
mi_global_fixed: 0.003286801278591156
rec_loss: 1.0049443244934082
kld_loss: 1.8239026069641113
mi_local_global: 0.001108326017856598
mi_global_fixed: 0.0017529800534248352
rec_loss: 1.0040555000305176
kld_loss: 1.8428845405578613
mi_local_global: 0.002055920660495758
m

2023-01-11 18:32:51,453 (laplacian_with_domain_separation_1MINE_VAE_Separated_client:144)INFO: {'Role': 'Client #1', 'Round': 0, 'Results_raw': {'train_total': 901, 'train_avg_loss': 10.833804, 'train_loss': 9761.257082, 'train_imp_ratio': -1072.108966}}


...

in val or test
mi_local_global: -0.022715583443641663
mi_global_fixed: -0.04148292914032936
rec_loss: 1.1096203327178955
kld_loss: 1.8284931182861328
mi_local_global: -0.024521291255950928
mi_global_fixed: -0.04162608087062836
rec_loss: 1.1148899793624878
kld_loss: 1.8832979202270508
in val or test
mi_local_global: -0.021364478394389153
mi_global_fixed: -0.03894304111599922
rec_loss: 1.1138468980789185
kld_loss: 1.8617722988128662
mi_local_global: -0.01919863373041153
mi_global_fixed: -0.03737098351120949
rec_loss: 1.1115999221801758
kld_loss: 1.8476321697235107
in val or test
mi_local_global: 0.02593022957444191
mi_global_fixed: -0.05254092440009117
rec_loss: 0.9481154680252075
kld_loss: 1.8367254734039307
in val or test


2023-01-11 18:34:38,732 (client:410)INFO: {'Role': 'Client #2', 'Round': 23, 'Results_raw': {'test_total': 64, 'test_avg_loss': 0.059721, 'test_loss': 3.822153, 'test_imp_ratio': -125.362772, 'val_total': 63, 'val_avg_loss': 0.068305, 'val_loss': 4.303208, 'val_imp_ratio': -157.754298}}
2023-01-11 18:34:38,733 (monitor:513)INFO: current_best=-136.479797, should_save=False


mi_local_global: 0.023067807778716087
mi_global_fixed: -0.0491296723484993
rec_loss: 0.9338778853416443
kld_loss: 1.7786040306091309
mi_local_global: 0.024055948480963707
mi_global_fixed: -0.04130414128303528
rec_loss: 0.9344663023948669
kld_loss: 1.7971546649932861
mi_local_global: 0.024444805458188057
mi_global_fixed: -0.04164976626634598
rec_loss: 0.9329872727394104
kld_loss: 1.7627544403076172
mi_local_global: 0.028619127348065376
mi_global_fixed: -0.04365919157862663
rec_loss: 0.9409197568893433
kld_loss: 1.8244476318359375
mi_local_global: 0.02082115411758423
mi_global_fixed: -0.04467354342341423
rec_loss: 0.9436658024787903
kld_loss: 1.820450782775879
mi_local_global: 0.021691501140594482
mi_global_fixed: -0.04613277688622475
rec_loss: 0.9330284595489502
kld_loss: 1.7752320766448975
mi_local_global: 0.022897202521562576
mi_global_fixed: -0.04269202798604965
rec_loss: 0.9454174637794495
kld_loss: 1.823815941810608
mi_local_global: 0.027855467051267624
mi_global_fixed: -0.03972215

2023-01-11 18:34:40,196 (laplacian_with_domain_separation_1MINE_VAE_Separated_client:144)INFO: {'Role': 'Client #2', 'Round': 23, 'Results_raw': {'train_total': 512, 'train_avg_loss': 0.095606, 'train_loss': 48.950337, 'train_imp_ratio': -260.777834}}


mi_local_global: 0.025191890075802803
mi_global_fixed: -0.047543637454509735
rec_loss: 0.93460613489151
kld_loss: 1.7970690727233887
mi_local_global: -0.01845431700348854
mi_global_fixed: -0.029514119029045105
rec_loss: 1.1062037944793701
kld_loss: 1.8194283246994019
mi_local_global: -0.017609916627407074
mi_global_fixed: -0.02789674513041973
rec_loss: 1.1050312519073486
kld_loss: 1.8258907794952393
mi_local_global: -0.01979362592101097
mi_global_fixed: -0.029292287304997444
rec_loss: 1.1146619319915771
kld_loss: 1.8703539371490479
mi_local_global: -0.018621394410729408
mi_global_fixed: -0.030657270923256874
rec_loss: 1.1087182760238647
kld_loss: 1.8331823348999023


client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [ ]:
777-431


In [ ]:
data

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']